In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import keras 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler, LabelEncoder
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
#for penalizing wrong weights heavily
from keras.regularizers import l2

preprocessing

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_dir = 'train/'
test_dir = 'test/'

#initialiing the test and train data modifications we need for preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255 )

#loading the testing and training data from the directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,

)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


model building


In [3]:
model =Sequential()

#our first layer should contain lesser neurons as to detect the major patterns and later on as the lareys becomes dense we increase the neurons
#to detect more complex paaterns

#layer1
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))

#layer 2
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
#we can apply max pooling after 1 or 2 convolutional layers in order to recude the diesnions and make is less prone to overfitting

#pool layer 1
#model.add(MaxPooling2D(pool_size=(2,2)))

#layer 3
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(64, (3, 3), activation='relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))

#to feed the data into fully connected dense layers we need to flatten it first so that we can apply dense layers on it
model.add(Flatten())

model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.4))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
#then we add the drop out layers to drop some part so that the model doen't overfit

model.add(Dense(32, activation='relu'))

#model.add(Dropout(0.4))
#last dense layer to predict the output class ,it have the nuerons same as the number of classes we have
#we will use the soft max for mmulti class classification as sigmoid is good for binary
model.add(Dense(7,activation='softmax'))

d:\machine_learning\project1\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# model =Sequential()

# #our first layer should contain lesser neurons as to detect the major patterns and later on as the lareys becomes dense we increase the neurons 
# #to detect more complex paaterns

# #layer1
# model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))

# #layer 2
# model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
# #we can apply max pooling after 1 or 2 convolutional layers in order to recude the diesnions and make is less prone to overfitting

# #pool layer 1
# model.add(MaxPooling2D(pool_size=(2,2)))

# #layer 3
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))

# # model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3), activation='relu'))

# # model.add(MaxPooling2D(pool_size=(2, 2)))

# #to feed the data into fully connected dense layers we need to flatten it first so that we can apply dense layers on it
# model.add(Flatten())

# model.add(Dense(128, activation='relu',kernel_regularizer=l2(0.01)))
# # model.add(Dropout(0.4))
# model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
# #then we add the drop out layers to drop some part so that the model doen't overfit

# model.add(Dense(128, activation='relu'))

# model.add(Dropout(0.4))
# #last dense layer to predict the output class ,it have the nuerons same as the number of classes we have
# #we will use the soft max for mmulti class classification as sigmoid is good for binary
# model.add(Dense(7,activation='softmax'))

In [5]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores model weights from the epoch with the best validation loss
)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#history to detect the losss functions
history = model.fit(
    train_generator,
    epochs=100,  # Set the number of epochs
    validation_data=test_generator,
    # callbacks=[early_stopping]
)

Epoch 1/100


d:\machine_learning\project1\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


449/449 ━━━━━━━━━━━━━━━━━━━━ 313s 685ms/step - accuracy: 0.2419 - loss: 2.2178 - val_accuracy: 0.3097 - val_loss: 1.7955
Epoch 2/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 150s 335ms/step - accuracy: 0.3142 - loss: 1.7480 - val_accuracy: 0.3853 - val_loss: 1.6220
Epoch 3/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 151s 335ms/step - accuracy: 0.4100 - loss: 1.5913 - val_accuracy: 0.4546 - val_loss: 1.4940
Epoch 4/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 151s 337ms/step - accuracy: 0.4631 - loss: 1.4778 - val_accuracy: 0.4656 - val_loss: 1.4732
Epoch 5/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 331s 737ms/step - accuracy: 0.4839 - loss: 1.4240 - val_accuracy: 0.4809 - val_loss: 1.4465
Epoch 6/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 15089s 34s/step - accuracy: 0.5031 - loss: 1.3876 - val_accuracy: 0.4816 - val_loss: 1.4415
Epoch 7/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 139s 309ms/step - accuracy: 0.5155 - loss: 1.3659 - val_accuracy: 0.4830 - val_loss: 1.4417
Epoch 8/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 128s 285ms/step - accuracy: 0.5254 - los

KeyboardInterrupt: 

In [6]:
history_frame = pd.DataFrame(history.history)

# Plot loss and val_loss
history_frame.loc[:, ['loss', 'val_loss']].plot()

# Plot accuracy and val_accuracy
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()
plt.show()


NameError: name 'history' is not defined